# BEE 4750 Homework 3: Dissolved Oxygen and Monte Carlo

**Name**:

**ID**:

> **Due Date**
>
> Thursday, 10/03/23, 9:00pm

## Overview

### Instructions

-   Problem 1 asks you to implement a model for dissolved oxygen in a
    river with multiple waste releases and use this to develop a
    strategy to ensure regulatory compliance.
-   Problem 2 asks you to use Monte Carlo simulation to assess how well
    your strategy from Problem 1 performs under uncertainty.
-   Problem 3 (5750 only) asks you to identify where a third discharge
    should be placed to maintain regulatory compliance.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
using Random
using Plots
using LaTeXStrings
using Distributions
using CSV
using DataFrames

## Problems (Total: 50/60 Points)

### Problem 1 (30 points)

A river which flows at 6 km/d is receiving waste discharges from two
sources which are 15 km apart. The oxygen reaeration rate is 0.55
day<sup>-1</sup>, and the decay rates of CBOD and NBOD are are 0.35 and
0.25 day<sup>-1</sup>, respectively. The river’s saturated dissolved
oxygen concentration is 10m g/L.

If the characteristics of the river inflow and waste discharges are
given in <a href="#tbl-river" class="quarto-xref">Table 1</a>, write a
Julia model to compute the dissolved oxygen concentration from the first
wastewater discharge to an arbitrary distance `d` km downstream. Use
your model to compute the maximum dissolved oxygen concentration up to
50km downstream and how far downriver this maximum occurs.

| Parameter | River Inflow | Waste Stream 1 | Waste Stream 2 |
|:--:|---:|---:|---:|
| Inflow | 100,000 m<sup>3</sup>/d | 10,000 m<sup>3</sup>/d | 15,000 m<sup>3</sup>/d |
| DO Concentration | 7.5 mg/L | 5 mg/L | 5 mg/L |
| CBOD | 5 mg/L | 50 mg/L | 45 mg/L |
| NBOD | 5 mg/L | 35 mg/L | 35 mg/L |

Table 1: River inflow and waste stream characteristics for Problem 1.

**In this problem**:

-   Plot the dissolved oxygen concentration from the first waste stream
    to 50m downriver. What is the minimum value in mg/L?
-   What is the minimum level of treatment (% removal of organic waste)
    for waste stream 1 that will ensure that the dissolved oxygen
    concentration never drops below 4 mg/L, assuming that waste stream 2
    remains untreated? How about if only waste stream 2 is treated?
-   Suppose you are responsible for designing a waste treatment plan for
    discharges into the river, with a regulatory mandate to keep the
    dissolved oxygen concentration above 4 mg/L. Discuss whether you’d
    opt to treat waste stream 2 alone or both waste streams equally.
    What other information might you need to make a conclusion, if any?

### Problem 1 Model Setup
#### Assumptions
- This river is a Prescriptive model because we are asked to specify, or prescribe, an action (in the form of a waste treatment plan) in order to optimize the river flow model to ensure its' DO concentration stays above 4 mg/L.
- Assume that Random, Plots, LaTexStrings, Distributions, CSV, and DataFrames packages may be utilized in hw03.
- Assume steady state waste in each section.
- Assume decay is linear with respect to mass.
- Assume deoxygenation from waste decomposition is 1st order with a rate k.
- For reaeration, assume a simple linear model based on difference between saturation DO concentration and DO concentration.
- Assume that it is permissible to ignore photosynthesis P_s, respiration R, and benthal uptake S_B in this simulated model.
- Assume C_s, when plotted, is a constant value.
- Assume the simulated output plot should have a shape similar to a "sag curve".
- Assume the system can be thought of as a multi-box system.
- The source begins on the left side at an assumed starting distance of 0 [km] (for model simplicity, i.e. enhanced and accurate analysis for the system)
    - Assume that the distance, d + 15, between Waste Stream 1 and Waste Stream 2, is also the distance from the river inflow to Waste Stream 1. 
        - Therefore, Waste Stream 1 occurs at a distance d = 15 [km]
- The problem statement section "In this problem:" states that the dissolved oxygen must be plotted from "the first waste stream to 50m downriver." However, it is reasonable to assume that the author of this problem intended for 'm' to actually be units of [km]. Therefore, in this model, a value of 50 [km] was used for the river flow simulation, and for the plot with a range of 0 [km] to 50 [km].

#### Constraints
- Steady state system.
- It is not permissible to assume a homogenous process for the simulation of this river flow model.
- Use fate and transport modeling approach to see how relevant quantities vary over arbitrary distance d within simulated river flow model.
- Track mass balance in terms of rates rather than absolute mass.
- Regulatory limit on DO, where DO > 4 [mg/L]
    - Interpretation: to ensure safety aquatic ecosystems and comply with environmental standards, specifically for the modeled river for Problem 1.
- River flow rate = 100000 [m^(3)/day], Waste Stream 1 flow rate = 10000 [m^(3)/day], and Waste Stream 2 flow rate = 15000 [m^(3)/day]
    - Cannot change for initial model simulation.
- Reaeration rate = 0.55 [day^(-1)], CBOD decay rate = 0.35 [day^(-1)], NBOD decay rate = 0.25 [day^(-1)], and saturation DO concentration = 10 [mg/L] 
     - Where the decay rate is assumed linear with respect to mass.
- Distance between Waste Stream 1 and Waste Stream 2 = 15 [km], total simulation distance = 50 [km]
     - Cannot change for initial model simulation.

#### Decision Variables
- B [mg/L]
- N [mg/L]
- N_ws1 [mg/L]
- N_ws2 [mg/L]
- B_ws1 [mg/L]
- B_ws2 [mg/L]

#### All Variables and their Definitions
- B [mg/L] = Carbonaceous Biochemical Oxygen Demand (CBOD) concentration
    - B is measured at a distance d downriver. Thus, B(d) is equivalent to the CBOD concentration at a specific distance d along the length of the river flow model. 
- B_decayed [mg/L] = Carbonaceous Biochemical Oxygen Demand (CBOD) concentration after accounting for external decay factors.
- N [mg/L] = Nitrogenous Biochemical Oxygen Demand (NBOD) concentration
    - N is measured at a distance d downriver. Thus, N(d) is equivalent to the NBOD concentration at a specific distance d along the length of the river flow model. 
- N_decayed [mg/L] = Nitrogenous Biochemical Oxygen Demand (NBOD) concentration after accounting for external decay factors.
- a1 = exponential oxygen reaeration rate 
    - a1 is measured over a distance of d [km].
- a2 = Oxygen Demand accommodating for CBOD
    - a2 is necessary because Oxygen Demand occurs due to oxygen consumption within the simulated river flow model based on the effects of CBOD.
    - a2 is measured over a distance of d.
- a3 = Oxygen Demand accommodating for NBOD
    - a3 is necessary because Oxygen Demand occurs due to oxygen consumption within the simulated river flow model based on the effects of NBOD. 
     - a3 is measured over a distance of d.
- C_f [mg/L] = DO concentration
    - C_f is measured at a distance d downriver. Thus, C_f(d) is equivalent to the DO concentration [mg/L] at a specific distance d along the length of the river flow model. 
    - C_f is based on the changes and values of other variables related to the effects of NBOD, CBOD, reaeration, and oxygen depletion.
- d [km] = specific distance along the total simulated river flow model length.
- C_r [mg/L] = DO concentration at a discharge distance (C_r(d)) in the total simulated river flow model length.
    - Initial condition: the initial concentration of DO [mg/L] for the river inflow.
- B_r [mg/L] = CBOD concentration at a discharge distance (B_r(d)) in the total simulated river flow model length.
    - the initial concentration of CBOD [mg/L] for the river inflow.
- N_r [mg/L] = NBOD concentration at a discharge distance (N_r(d)) in the total simulated river flow model length.
    - Initial condition: the initial concentration of NBOD [mg/L] for the river inflow.
- ka [day^(-1)]= raeration rate 
    - ka is a constant which represents the oxygen that is added to the water in the simulated river flow model, and is added at the specified rate.
- kc [day^(-1)]= CBOD decay rate 
    - kc is a constant which represents the oxygen that removed from the water in the form of decay in the simulated river flow model, accounting for the effects of CBOD, and this decay occurs at the specified rate.
- kn [day^(-1)]= NBOD decay rate 
    - kn is a constant which represents the oxygen that removed from the water in the form of decay in the simulated river flow model, accounting for the effects of NBOD, and this decay occurs at the specified rate.
- U [km/day] = river velocity
- discharge_dist_ws1 [km] = distance at which Waste Stream 1 is discharged into the river
- discharge_dist_ws2 [km] = distance at which Waste Stream 1 is discharged into the river
- C_i [mg/L] = DO concentration at a discharge distance (C_i(d)) in the total simulated river flow model length after accommodating for mixing 
- Q [m^(3)/day] = total flow rate
- C_min [mg/L] = minimum observed DO concentration during simulation of the river flow model
##### Variable Initial Conditions Defined
According to the BEE 4750 HW03 Problem 1 Statement, the initial conditions for the following variables are defined:
- C_r [mg/L] = 7.5 [mg/L]
    - The DO concentration in river inflow 
- C_ws1 [mg/L] = 5 [mg/L]
    - The DO concentration in Waste Stream 1 source
- C_ws2 [mg/L] = 5 [mg/L]
    - The DO concentration in Waste Stream 2 source
- B_r [mg/L] = 5 [mg/L]
    - The CBOD concentration in river inflow 
- B_ws1 [mg/L] = 50 [mg/L]
    - The CBOD concentration in Waste Stream 1 source
- B_ws2 [mg/L] = 45 [mg/L]
    - The CBOD concentration in Waste Stream 2 source
- N_r [mg/L] = 5 [mg/L]
    - The NBOD concentration in river inflow 
- N_ws1 [mg/L] = 35 [mg/L]
    - The NBOD concentration in Waste Stream 1 source
- N_ws2 [mg/L] = 35 [mg/L]
    - The CBOD concentration in Waste Stream 2 source
- U [km/day] = 6 [km/day] 
    - The velocity of the river inflow
- ka [day^(-1)] = 0.55 [day^(-1)]
    - The reaeration rate constant
- kn [day^(-1)] = 0.35 [day^(-1)]
    - The NBOD first order decay rate constant
- kc [day^(-1)] = 0.25 [day^(-1)]
    - The CBOD first order decay rate constant
- Cs [mg/L] = 10 [mg/L]
    - The river's DO saturation concentration (maximized oxygen amount that can dissolve into the river water in the simulated flow model).

##### Derivation for Objective 
- **Objective:** The primary goal of the Problem 1 model is to derive and implement a model for determining the dissolved oxygen (DO) concentration in a river at various distances, accounting for 2 Waste Stream sources that discharge multiple waste types. Then, use this to develop a strategy to ensure regulatory compliance for keeping the regulatory limit on DO concentration above 4 [mg/L]. On a broader environmental level, it is necessary to ensure the regulatory limit on DO concentration above 4 [mg/L] is upheld so that aquatic ecosystems with organisms and respective habitats can survive, and hypoxia does not occur.
    - Component 1: The initial river indflow DO concentration is simulated and can be influenced by CBOD or NBOD concentrations that are in the water at the beginning of the modeled distance. All three concentrations are given as input parameters, and are implemented in later model components.
    - Component 2: Initialize two Waste Stream sources at the locations d = 15 [km] (Waste Stream 1) and d + 15 [km] (Waste Stream 2). Account for the DO, CBOD, and NBOD concentration from  Waste Stream 1 at distance d, onwards through the length of the river. Also account for the DO, CBOD, and NBOD concentration from Waste Stream 2, starting at a distance of d + 15 [km] to analyze how the variations of CBOD and NBOD can affect DO depletion in the river flow model. 
    - Component 3: Implement the effects of oxygen reaeration on the system as it changes with distance, which involved atmospheric oxygen being reabsorbed into the river water. The rate at which oxygen reaeration occurs is proportional to the difference between the DO saturation concentration [mg/L] and the observed DO concentration [mg/L] at each simulation step-size (where step size = 1 [km]). 
- **Goal:**
    - The overarching objective of this simulation is to simulate the river flow model to determine DO concentration at an arbitrary distance d [km] downriver and the minimum DO concentration over the simulated river flow model.
    - Plot the DO concentration from the first waste stream to 50 [km] downriver, find its minimum value [mg/L]. Find the minimum level of treatment for Waste Stream 1 to ensure that DO concentration never falls below 4 mg/L, assuming Waste Stream 2 continues not to be treated, and the opposite case as well. 
    - Develop a strategy for ensuring the DO concentration of the river does not follow below the regulatory limit of 4 [mg/L] based on the results of Component 4. Furthermore, discuss how the results would influence a choice between potential waste water treatment plans for the river. 
- **Steps to achieve Objective**:
    - Derive equations for DO concentration [mg/L] as a function of distance for the river flow model.
    - Simulate the model over a distance up to 50 [km].
    - Plot the DO concentration [mg/L] and identify distances at which treatment may be needed to ensure regulatory compliance of keeping the DO concentration above 4 [mg/L].

##### Mathematical Derivation for Objective
- **CBOD and NBOD decay functions:**
    - CBOD river concentration(distance d [km]) [mg/L] = CBOD inflow concentration [mg/L] * e^((-CBOD decay rate * distance d [km])/(river flow velocity [km/day]))
    - NBOD river concentration(distance d [km]) [mg/L] = NBOD inflow concentration [mg/L] * e^((-NBOD decay rate * distance d [km])/(river flow velocity [km/day]))
        - Intepretation of CBOD and NBOD decay function: accounts for the exponential decay of CBOD and NBOD to calculate CBOD and NBOD concentrations at a distance d in the river flow model. 
- **Depletion of DO equation:**
    - d(DO depletion(distance d [km])) = CBOD_river(distance d [km]) [mg/L] + NBOD_river(distance d [km]) [mg/L] 
        - Intepretation: calculates DO depletion at each simulated step size of 0.05 [km], accounting for CBOD and NBOD decay.
- **Reaeration of DO equation:**
    - d(DO reaeration(distance d [km])) = reaeration rate * (saturation DO concentration [mg/L] - observed DO concentration(distance d [km]) [mg/L]) * ((1 - e^(-reaeration rate * distance d [km])))
        - Interpretation: DO replenishing rate in the river water.
- **Mass balance for variations in DO concentration in the river:**
    - dDO/dD = (((-CBOD decay rate * CBOD conentration(distance d [km])) + (-NBOD decay rate * NBOD conentration(distance d [km]))) / (river flow rate [m^(3)/day] + Waste Stream 1 flow rate [m^(3)/day] + Waste Stream 2 flow rate [m^(3)/day])) + (reaeration rate * (saturation DO concentration [mg/L] - observed DO concentration(distance d [km]) [mg/L]) * (1 - e^(-reaeration rate * distance d [km])))
        - Interpretation of mass balance for variations in DO concentration in the river derivation: this mass balance allows me, the waste treatment plan designer, to evaluate the variations in DO concentrations with distance downriver, considering the effects of oxygen depletion through CBOD and NBOD decay, as well as the oxygen reaeration rate that occurs proportionally with the difference between the DO saturated and current DO concentrations.
- **Mass balance for  DO concentration at a distance D:**
    - DO concentration(distance d [km]) [mg/L] = Initial DO concentration [mg/L] - d(DO depletion(distance d [km])) + d(DO reaeration(distance d [km]))
        - Where Initial DO concentration [mg/L] = initial DO concentration [mg/L] before Waste Stream 1 inflow, and DO concentration(distance d [km]) [mg/L] is the concentration observed at a distance in the length of the simulated river.
- **Mass balance for final DO concentration**:
    - DO final concentration (d) = (river DO concentration [mg/L]* river flow rate [m^3/day] + Waste Stream 1 DO concentration [mg/L]* Waste Stream 1 flow rate [m^3/day] + Waste Stream 2 DO concentration [mg/L]* Waste Stream 2 flow rate [m^3/day])/((river flow rate [m^(3)/day] + Waste Stream 1 flow rate [m^(3)/day] + Waste Stream 2 flow rate [m^(3)/day]))

#### Minimum Treatment
- Oxygen depletion:
    - D [mg/L] = Cs [mg/L] - C_r [mg/L]
    - D [mg/L] = 10 [mg/L] - 7.5 [mg/L]
    - D [mg/L] = 2.5 [mg/L]
- Maximum oxygen depletion:
    - Maximum D [mg/L] = Cs [mg/L] - C_rs [mg/L]
    - Maximum D [mg/L] = 10 [mg/L] - 4 [mg/L]
    - Maximum D [mg/L] = 6 [mg/L]
- TOD
    - Waste Stream 1 source:
        - TOD_ws1 [mg/L] = CBOD Concentration_ws1 [mg/L] + NBOD Concentration_ws1 [mg/L]
        - TOD_ws1 [mg/L] = 50 [mg/L] + 35 [mg/L]
        - TOD_ws1 [mg/L] = 85 [mg/L]
    - Waste Stream 2 source:
        - TOD_ws2 [mg/L] = CBOD Concentration_ws2 [mg/L] + NBOD Concentration_ws2 [mg/L]
        - TOD_ws2 [mg/L] = 45 [mg/L] + 35 [mg/L]
        - TOD_ws2 [mg/L] = 80 [mg/L]
    - Maximum TOD [mg/L] = Maximum D [mg/L] - D [mg/L]
        - Maximum TOD [mg/L] = 6 - 2.5 [mg/L]
        - Maximum TOD [mg/L]= 3.5 [mg/L]

- Minimum treatment percentage, Waste Stream 1:
    - Minimum treatment required_ws1 = ((TOD_ws1 - Maximum TOD)/(TOD_ws1)) * 100
    - Minimum treatment required_ws1 [%] = ((85 - 3.5)/(85)) * 100
    - Minimum treatment required_ws1 [%] = 95.88%
- Minimum treatment percentage, Waste Stream 2:
    - Minimum treatment required_ws2 = ((TOD_ws2 - Maximum TOD)/(TOD_ws2)) * 100
        - Minimum treatment required_ws2 [%] = ((80 - 3.5)/(80)) * 100
        - Minimum treatment required_ws2 [%] = 95.63%

##### Final Answer
- The minimum concentration of DO in the river is C_min = 3.0261584628034104 [mg/L]
- The minimum level of treatment (% removal of organic waste) for Waste Stream 1 that will ensure the DO concentration never drops below 4 mg/L, assuming Waste Stream 2 remains untreated = 95.88%
- The minimum level of treatment (% removal of organic waste) for Waste Stream 2 that will ensure the DO concentration never drops below 4 mg/L, assuming Waste Stream 1 remains untreated = 95.63%

##### Specific Reference to Modeling Results
- **Step-size choice:** Chose a step size of 1 [km] for d = 0:1:50 # [km]. A step size of 1 [km] was chosen to provide a highly comprehensive, thorough representation of the river flow model to show how the DO concentration varies with distance from 0 to 50 km in the river.
- **Minimum DO concentration and plot discussion:** 
    - Based on the output of the model, as detailed in the Final Answer section, the minimum DO concentration is C_min = 3.0261584628034104, which signals that the regulatory limit of keeping DO concentration above 4 mg/L was not upheld during the simulation.
    - The plot of Dissolved Oxygen Concentration Variations over River Distance, with an independent variable Distance d [km] and a dependant variable, DO Concentration [mg/L] presents an interesting representation of the river flow model. First, the river inflow experiences a slight decrease in DO concentration before experiencing reaeration. At a distance of 15 km, Waste Stream 1 source flows into the river, during which a sharp decrease in DO concentration occurs before reaeration overtakes oxygen depletion, allowing for an increase until 30 km. At 30 km, Waste Stream 2 source flows into the river, the point at which a sharp decrease in DO concentration occurs again due to oxygen depletion, before then once again increasing as reaeration overtakes the oxygen depletion. The increasing DO concentration continues over the remainder of the distance d of the river flow model, up to 50 km. The significance of steep increases and decreases provides insights into the adjustment period for the river when accounting for Waste Stream source inflow DO, NBOD, and CBOD concentrations. 
- **Rectification based on modeling results:** 
    - It would be wise to investigate the possibility of changing factors which influence DO in order to protect water quality and aquatic life, prevent hypoxia, and ensure the regulatory limit of >4 [mg/L] is upheld. For instance, some factors influencing DO include organic waste decomposition, nitrification, temperature, and pressure. For this waste treatment plan design, a treatment consisting of temperature decrease could be utilized in Waste Stream 1 and Waste Stream 2 in order to ensure that DO concentration stays above 4 [mg/L]. According to The United States Environmental Protection Agency (https://www.epa.gov/caddis/dissolved-oxygen), higher temperatures decrease oxygen solubility in water, which then lessens the amount of DO that can be held by the water. This reinforcing feedback means that when the opposite is done, (i.e. decreasing the river inflow, Waste Stream 1, and Waste Stream 2 water temperatures), the solubility is thereby increased, which increases the amount of DO that can be held by the water. An additional chemical treatment would still be necessary to prevent CBOD and NBOD decay in Waste Stream 1 and 2 inlets, as the DO concentration may not accumulate enough by the respective inlets to allow for the regulatory limit of >4 [mg/L] to be upheld. 
    - For the chemical treatment for this river flow model, it is most essential to treate Waste Stream 1 and Waste Stream 2 approximately equally since the minimum treatment required for Waste Stream 1, with no Waste Stream 2 treatment [%] = 95.88%, and the minimum treatment required for Waste Stream 2, with no Waste Stream 1 treatment [%] = 95.63%. The percentage values for both Waste Stream 1 and Waste Stream 2 are within .25% of one another, so they should be treated approximately equally. Additionally, since the minimum percentage of required treatment is high (close to 100%) for both Waste Stream 1 and Waste Stream 2, it is necessary to chemically treat both sources for a large reduction in Total Oxygen Demand required to align with the Maximum Oxygen Demand value of 6 mg/L. For instance, the Total Oxygen Demand for Waste Stream 1 is 85 mg/L, which requires a large reduction through chemical treatment. Similarly, the Total Oxygen Demand for Waste Stream 2 is 80 mg/L, which also requires a large reduction through chemical treatment.
- To make a final conclusion, this simulation should be applied based on real-world, empirical data for a specified river that has 2 Waste Stream sources. The use of empirical data allows for a more thorough, comprehensive, and accurate analysis for specific sites. 

In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

using Random
using Plots
using LaTeXStrings
using Distributions
using CSV
using DataFrames

# Initialize function to calculate DO based on relevant decision variables
function do_simulate(d, velocity_r, velocity_ws1, velocity_ws2, C_r, C_ws1, C_ws2, B_r, B_ws1, B_ws2, N_r, N_ws1, N_ws2, ka, kn, kc, Cs, U, 
    discharge_dist_ws1, discharge_dist_ws2)

    # Initialize conditions for distance d along simulated river flow model, separate by river inflow, Waste Stream 1 source, and Waste Stream 2 source
    # For when d hasn't reached Waste Stream 1 source
    if d < discharge_dist_ws1
        # Calculate flow, no sum, just river velocity since Waste Stream sources haven't begun 
        Q = velocity_r # [m^(3)/day]

        # Calculate concentrations of DO, CBOD, and NBOD, just for river conditions since Waste Stream sources haven't begun
        C_i = C_r
        B = B_r
        N = N_r

    # After Waste Stream 1 source and before Waste Stream 2 discharge source
    elseif d < discharge_dist_ws2
        # Calculate flow, sum respective river and Waste Stream 1 velocitys
        Q = velocity_r + velocity_ws1 # [m^(3)/day]

        # Calculate concentrations of DO, CBOD, and NBOD based on river and Waste Stream 1 sources over total flow over d < discharge_dist_ws2
        C_i = (C_r * velocity_r + C_ws1 * velocity_ws1) / Q
        B = (B_r * velocity_r + B_ws1 * velocity_ws1) / Q
        N = (N_r * velocity_r + N_ws1 * velocity_ws1) / Q

    # Waste Stream 2 source to end of distance d along river
    else
        # Calculate flow, sum of all inflow source velocitys
        Q = velocity_r + velocity_ws1 + velocity_ws2 # [m^(3)/day]

        # Calculate concentrations of DO, CBOD, and NBOD now for all 3 inflow sources
        C_i = (C_r * velocity_r + C_ws1 * velocity_ws1 + C_ws2 * velocity_ws2) / Q
        B = (B_r * velocity_r + B_ws1 * velocity_ws1 + B_ws2 * velocity_ws2) / Q
        N = (N_r * velocity_r + N_ws1 * velocity_ws1 + N_ws2 * velocity_ws2) / Q
    end
    
    # Calculate CBOD and NBOD concentrations, account for exponential decay
    # Waste Stream 2 source CBOD and NBOD concentration accounting for decay
    if d >= discharge_dist_ws2
        B_decayed = B * exp(-kc * (d - discharge_dist_ws1) / U) + B * exp(-kc * (d - discharge_dist_ws2) / U)
        N_decayed = N * exp(-kn * (d - discharge_dist_ws1) / U) + N * exp(-kn * (d - discharge_dist_ws2) / U)
        
    # Waste Stream 1 source CBOD and NBOD concentration accounting for decay
    elseif d >= discharge_dist_ws1
        B_decayed = B * exp(-kc * (d - discharge_dist_ws1) / U)
        N_decayed = N * exp(-kn * (d - discharge_dist_ws1) / U)
    
    # River inflow CBOD and NBOD concentration before Waste Stream 1 source with no decay
    else
        B_decayed = B
        N_decayed = N
end

    # Calculate DO, after accounting for oxygen reaeration
    a1 = exp(-ka * d / U) 
    a2 = (kc / (ka - kc)) * (exp(-kc * (d / U)) - exp(-ka * (d / U)))
    a3 = (kn / (ka - kn)) * (exp(-kn * (d / U)) - exp(-ka * (d / U)))

    # Calculate final DO concentration and return concentrations
    C_f = Cs * (1 - a1) + (C_i * a1) - (B_decayed * a2) - (N_decayed * a3)
    return C_f, B_decayed, N_decayed
end

# Set river and waste stream properties
velocity_r = 100000 # [m^(3)/day]
velocity_ws1 = 10000 # [m^(3)/day]
velocity_ws2 = 15000 # [m^(3)/day]

# DO concentrations
C_r = 7.5 # [mg/L]
C_ws1 = 5 # [mg/L]
C_ws2 = 5 # [mg/L]

# CBOD concentrations
B_r = 5 # [mg/L]
B_ws1 = 50 # [mg/L]
B_ws2 = 45 # [mg/L]

# NBOD concentrations
N_r = 5 # [mg/L]
N_ws1 = 35 # [mg/L]
N_ws2 = 35 # [mg/L]

# Model parameters
ka = 0.55 # [day^(-1)]
kc = 0.35 # [day^(-1)]
kn = 0.25  # [day^(-1)]
Cs = 10 # [mg/L]
U = 6 # [km/day]

# Initialize Waste Stream 1 and Waste Stream 2 source distances
discharge_dist_ws1 = 15 # [km]
discharge_dist_ws2 = discharge_dist_ws1 + 15 # [km]

# Initialize arbitrary distance to be length of river flow model
d = 0:1:50 # [km]

# Simulate full river flow model
do_simulated = [do_simulate(y, velocity_r, velocity_ws1, velocity_ws2,  C_r, C_ws1, C_ws2, 
    B_r, B_ws1, B_ws2, N_r, N_ws1, N_ws2, ka, kn, kc, Cs, U, discharge_dist_ws1, 
    discharge_dist_ws2) for y in d]

# Initialize arrays for DO, CBOD, and NBOD concentrations to plot simulated results
C_f = [d[1] for d in do_simulated]
B = [d[2] for d in do_simulated]
N = [d[3] for d in do_simulated]

# Calculate minimum DO concentration observed, plot as line later
C_min = minimum([d[1] for d in do_simulated])
return C_min

# Plot DO, CBOD, and NBOD concentrations after simulation iteration
using Plots
p1 = plot(d, C_f, ylabel="DO Concentration [mg/L]", xlabel="Distance [km]", label="DO", linewidth=3, color=:black, title="DO Concentration Variations over River Distance")
plot!(p1, d, B, label="CBOD Concentration [mg/L]", linestyle=:dash, linewidth=3, color=:green)
plot!(p1, d, N, label="NBOD Concentration [mg/L]", linestyle=:dash, linewidth=3, color=:blue)

# Plot DO concentration regulatory standard and minimum DO concentration value
hline!([4], color=:green, label="Regulatory Standard: DO > 4 [mg/L]") # Green, follows regulatory standard
hline!([C_min], color=:red, label="Minimum DO Concentration [mg/L]") # Red, below regulatory standard
display(p1)

### Problem 2 (20 points)

The simplest climate model involves capturing changes to the Earth’s
energy budget (it is commonly called the *energy balance model*, or
EBM). These changes are also called *radiative forcings* (RF), and can
result from several causes, including greenhouse gas emissions, volcanic
eruptions, and changes to the solar cycle. The EBM treats the Earth as a
0-dimensional sphere covered with water, which absorbs heat in response
to radiative forcings. Chanwith global temperature changes resulting
from imbalances in the average (over the entire surface area) heat flux.

The EBM equations are:

$$
\begin{align*}
\overbrace{\frac{dH}{dt}}^{\text{change in heat}} &= \overbrace{F}^{\substack{\text{radiative} \\ \text{forcing}}} - \overbrace{\lambda T}^{\substack{\text{change in} \\ \text{temperature}}} \\
\underbrace{C}_{\substack{\text{ocean heat} \\ \text{capacity}}} \frac{dT}{dt} &= F - \lambda T \\
c\underbrace{d}_{\substack{\text{ocean} \\ \text{mixing depth}}} \frac{dT}{dt} &= F - \lambda T,
\end{align*}
$$

where $c = 4.184\times 10^6 \mathrm{J/K/m}^2$ is the specific heat of
water per area, $d$ is the depth of the ocean mixed layer (we’ll assume
$d = 86 \mathrm{m}$), and $\lambda$ is the **climate feedback factor**
and controls how much the Earth warms in response to increased radiative
forcing (assume
$\lambda = 2.1^\circ \mathrm{C}/(\mathrm{W}/\mathrm{m}^2$)). The total
radiative forcing $F = F_\text{non-aerosol} + \alpha F_\text{aerosol}$,
where $\alpha$ is an uncertain scaling factor reflecting aerosol-cloud
feedbacks (we’ll assume $\alpha = 0.8$).

The code below loads historical and projected radiative forcings (under
the SSP5-8.5 future emissions scenario, which is the most extreme of the
scenarios used to project climate change impacts) from
`data/ERF_ssp585_1750-2500.csv` into a `DataFrame` object and calculates
the non-aerosol and aerosol components of those forcings.

> **Tip**
>
> Look closely at and experiment with the code below: `DataFrames` are a
> common Julia datatype for tabular data, and you may work more with
> them later in the semester or beyond! They are broadly similar to
> `DataFrames` from `pandas` in Python.

In [ ]:
# Dataset from https://zenodo.org/record/3973015
# The CSV is read into a DataFrame object, and we specify that it is comma delimited
forcings_all = CSV.read("data/ERF_ssp585_1750-2500.csv", DataFrame, delim=",")

# Separate out the individual components
# Get total aerosol forcings
forcing_aerosol_rad = forcings_all[!,"aerosol-radiation_interactions"]
forcing_aerosol_cloud = forcings_all[!,"aerosol-cloud_interactions"]
forcing_aerosol = forcing_aerosol_rad + forcing_aerosol_cloud
# Calculate non-aerosol forcings from the total.
forcing_total = forcings_all[!,"total"]
forcing_non_aerosol = forcing_total - forcing_aerosol

We can plot the aerosol and non-aerosol forcings below.

In [ ]:
t = Int64.(forcings_all[!,"year"]) # Ensure that years are interpreted as integers
p_forcing = plot(; xlabel="Year", ylabel="Radiative Forcing (W/m²)")
plot!(p_forcing, t, forcing_aerosol, label="Aerosol Forcing", color=:blue, linewidth=2)
plot!(p_forcing, t, forcing_non_aerosol, label="Non-Aerosol Forcing", color=:red, linewidth=2)

**In this problem**:

-   Discretize the EBM to produce a simulation model of global mean
    temperatures $T$ over time as a result of total radiative forcings
    $F$.

-   Simulate global mean temperature anomalies (in $^\circ C$ relative
    to 1750) from your model using the historical and SSP5-8.5 radiative
    forcing data. Use an annual time step for the simulation (in
    seconds: $\Delta t = 31,558,152 \mathrm{s}$). You can assume
    $T(0) = 0^\circ C$. Plot the resulting temperature simulation.

-   The climate feedback factor $\lambda$ is one of the key
    uncertainties in projecting future temperatures, even assuming a
    particular scenario of future radiative forcing. Suppose we use the
    following distribution for $\lambda$,
    $$\lambda \sim \text{LogNormal}(\log(2.1), \log(2)/4).$$

    Use Monte Carlo simulation to estimate the expected temperature in
    2100 assuming SSP5-RCP 8.5 radiative forcings (with 95% confidence
    intervals). How does this estimate compare to the value you got from
    your simulation using the expected value of
    $\lambda=2.1^\circ \mathrm{C}/(\mathrm{W}/\mathrm{m}^2)$? How did
    you decide your sample set was sufficient?

### Problem 3 (10 points)

**This problem is only required for students in BEE 5750**.

A factory is planning a third wastewater discharge into the river
downstream of the second plant. This discharge would consist of 5
m<sup>3</sup>/day of wastewater with a dissolved oxygen content of 4.5
mg/L and CBOD and NBOD levels of 50 and 45 mg/L, respectively.

**In this problem**:

-   Assume that the treatment plan you identified in Problem 1 is still
    in place for the existing discharges. If the third discharge will
    not be treated, under the original inflow conditions (7.5 mg/L DO),
    how far downstream from the second discharge does this third
    discharge need to be placed to keep the river concentration from
    dropping below 4 mg/L?

## References

List any external references consulted, including classmates.